# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-10-04'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('GC=F',start_date, today).reset_index()

btc_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1943,2023-09-27,"$1,871.60","$1,871.60","$1,871.60","$1,871.60","$1,871.60",12251
1944,2023-09-28,"$1,875.50","$1,878.20","$1,856.70","$1,860.40","$1,860.40",4654
1945,2023-09-29,"$1,864.30","$1,872.30","$1,846.00","$1,848.10","$1,848.10",1605
1946,2023-10-02,"$1,844.10","$1,845.90","$1,826.70","$1,830.00","$1,830.00",2309
1947,2023-10-03,"$1,825.00","$1,830.90","$1,816.80","$1,824.60","$1,824.60",2309


In [ ]:
# TAREA: Validamos que no hay datos vacíos
btc_df.isnull().sum()
# asignar una categoría única a los nulos, por ejemplo reemplazarlos por la media.

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date','Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
new_names = {
    "Date": "ds",
    "Open": "y",
}

# Los renombramos con los nuevos nombres
df.rename(columns=new_names, inplace=True)
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1943,2023-09-27,"$1,871.60"
1944,2023-09-28,"$1,875.50"
1945,2023-09-29,"$1,864.30"
1946,2023-10-02,"$1,844.10"
1947,2023-10-03,"$1,825.00"


In [ ]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de Bitcoin Precio de Apertura",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative" nombre_variable = Modelo()
m = Prophet()

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df nombre_variable.fit(datos)
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9suafde8/7buva4u7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9suafde8/yvdapo72.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97798', 'data', 'file=/tmp/tmp9suafde8/7buva4u7.json', 'init=/tmp/tmp9suafde8/yvdapo72.json', 'output', 'file=/tmp/tmp9suafde8/prophet_modelltgycn3w/prophet_model-20231004104259.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:42:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:43:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2308,2024-09-28
2309,2024-09-29
2310,2024-09-30
2311,2024-10-01
2312,2024-10-02


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
2308,2024-09-28
2309,2024-09-29
2310,2024-09-30
2311,2024-10-01


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2861,2023-11-01,"$-10,825.41","$-39,682.86","$17,938.76"
2862,2023-11-02,"$-10,961.69","$-40,461.14","$16,686.57"
2863,2023-11-03,"$-11,038.14","$-40,444.04","$17,039.96"
2864,2023-11-04,"$-11,148.12","$-41,036.03","$17,486.87"
2865,2023-11-05,"$-11,260.74","$-42,317.68","$16,268.09"


In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

1864.3416782387942

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)